# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
cities = pd.read_csv("cities.csv", encoding="utf-8")
cities.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Lompoc,20,US,1599962858,87,34.64,-120.46,64.00,6.93
1,Mataura,100,NZ,1599963086,97,-46.19,168.86,48.00,1.99
2,Severo-Kuril'sk,83,RU,1599963087,79,50.68,156.12,51.22,22.48
3,Thompson,75,CA,1599963087,80,55.74,-97.86,41.00,5.82
4,Cabo San Lucas,40,MX,1599963065,78,22.89,-109.91,82.99,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = cities["Humidity"]

In [24]:
fig = gmaps.figure(center=(40.0, -4.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity).astype(float), point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
narrowed_city_df = cities.loc[(cities["Wind Speed"] <= 10) & (cities["Cloudiness"] == 0) & \
                                   (cities["Max Temp"] >= 70) & (cities["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
43,Lagoa,0,PT,1599963099,73,39.05,-27.98,70.57,6.13
246,Caravelas,0,BR,1599963154,85,-17.71,-39.25,71.29,6.24
251,Tūkrah,0,LY,1599963156,63,32.53,20.58,73.63,6.31
286,Suez,0,EG,1599963165,84,29.97,32.53,74.32,9.60
367,Arrondissement de Bastia,0,FR,1599963187,68,42.67,9.33,75.99,4.70
381,Mizdah,0,LY,1599963191,76,31.45,12.98,71.44,7.99
400,Carutapera,0,BR,1599963195,85,-1.20,-46.02,78.66,7.70
406,Marsá Maţrūḩ,0,EG,1599963197,79,31.35,27.25,73.53,1.52
426,Zuwārah,0,LY,1599963202,75,32.93,12.08,77.47,3.89
447,Mabaruma,0,GY,1599963208,88,8.20,-59.78,77.74,4.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Lagoa,PT,39.05,-27.98,
246,Caravelas,BR,-17.71,-39.25,
251,Tūkrah,LY,32.53,20.58,
286,Suez,EG,29.97,32.53,
367,Arrondissement de Bastia,FR,42.67,9.33,
381,Mizdah,LY,31.45,12.98,
400,Carutapera,BR,-1.20,-46.02,
406,Marsá Maţrūḩ,EG,31.35,27.25,
426,Zuwārah,LY,32.93,12.08,
447,Mabaruma,GY,8.20,-59.78,


In [27]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [28]:
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

print("-------End of Search-------")

Retrieving Results for Index 43: Lagoa.
Closest hotel in Lagoa is Graciosa Resort - Biosphere Island Hotel.
------------
Retrieving Results for Index 246: Caravelas.
Closest hotel in Caravelas is Pousada Encanto Abrolhos.
------------
Retrieving Results for Index 251: Tūkrah.
Closest hotel in Tūkrah is قاعدة فيقا فيقا.
------------
Retrieving Results for Index 286: Suez.
Closest hotel in Suez is Green House.
------------
Retrieving Results for Index 367: Arrondissement de Bastia.
Closest hotel in Arrondissement de Bastia is Hôtel La Florentine.
------------
Retrieving Results for Index 381: Mizdah.
Closest hotel in Mizdah is فندق مزدة الكبير.
------------
Retrieving Results for Index 400: Carutapera.
Closest hotel in Carutapera is HOTEL R. SOARES.
------------
Retrieving Results for Index 406: Marsá Maţrūḩ.
Closest hotel in Marsá Maţrūḩ is Adriatica Hotel.
------------
Retrieving Results for Index 426: Zuwārah.
Closest hotel in Zuwārah is Zuwara hotel - فندق زوارة.
------------
Retriev

In [29]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Lagoa,PT,39.05,-27.98,Graciosa Resort - Biosphere Island Hotel
246,Caravelas,BR,-17.71,-39.25,Pousada Encanto Abrolhos
251,Tūkrah,LY,32.53,20.58,قاعدة فيقا فيقا
286,Suez,EG,29.97,32.53,Green House
367,Arrondissement de Bastia,FR,42.67,9.33,Hôtel La Florentine
381,Mizdah,LY,31.45,12.98,فندق مزدة الكبير
400,Carutapera,BR,-1.20,-46.02,HOTEL R. SOARES
406,Marsá Maţrūḩ,EG,31.35,27.25,Adriatica Hotel
426,Zuwārah,LY,32.93,12.08,Zuwara hotel - فندق زوارة
447,Mabaruma,GY,8.20,-59.78,


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))